Importing Modules

In [1]:
import pandas as pd
import numpy as np
import scipy
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

import datetime

import chart_studio.plotly as py
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
import colorlover as cl
from plotly.subplots import make_subplots
from scipy.stats import ttest_ind, pearsonr, linregress
from scipy.optimize import curve_fit
import kaleido
import os as os

from sklearn.metrics import r2_score 

from statsmodels.formula.api import ols
from statsmodels.stats.anova import anova_lm

import math as math

In [2]:
def LinRegress(X, Y):
    data = pd.DataFrame({'x':X, 'y': Y})
    model = ols("y ~ x", data).fit()
    return model

def exponenial_func(x, a, b, c):
    return a*np.exp(-b*x)+c

def linear_func(x, b,m):
    return (m*x)+b

In [3]:
####     COLOR VALUES     ####

red = 'rgb(128, 0, 0)'
light_red = 'rgb(255, 153, 153)'

light_blue = 'rgb(135,206,235)'

Reading data

In [4]:
df = pd.read_csv('CSV/Mean House Prices/MSPUS.csv')

copy = df.copy() # backup of dataset

df.isna().sum() 

DATE     0
MSPUS    0
dtype: int64

In [5]:
df.head()

,DATE,MSPUS
0,1963-01-01,17800.0
1,1963-04-01,18000.0
2,1963-07-01,17900.0
3,1963-10-01,18500.0
4,1964-01-01,18500.0


section 1 :D

In [42]:
df.dropna(axis = 0, inplace = True) # delete all rows that have N/A values

df1 = df.copy()

df1['year'] = pd.to_datetime(df1['DATE']).dt.year
df1.drop(columns = ['DATE'], axis = 1, inplace = True)

year_avg = df1.groupby('year')[['MSPUS']].mean().reset_index()

year_avg.tail()

,year,MSPUS
57,2020,328150.0
58,2021,383000.0
59,2022,432950.0
60,2023,426525.0
61,2024,419550.0


In [13]:
fig = go.Figure()
fig.update_layout(title="Temporal House Price Over Quarters: 1963 - 2024", title_font_size = 20,
                  font=dict( family="Courier New, monospace", size=12,color="#7f7f7f"),
                  template = "ggplot2", hovermode= 'closest')
fig.update_xaxes(showline=True, linewidth=1, linecolor='gray')
fig.update_yaxes(showline=True, linewidth=1, linecolor='gray')

fig.add_trace(go.Scatter(x = df['DATE'], y = df['MSPUS'], mode = 'lines',
                         name = 'Avg. Temp', marker_color=red))

# ####     REGRESSION     ####
# 
# X = pd.DataFrame(avg_abs[avg_abs['Year'].between(1850,1975)][['Year']])
# y = pd.DataFrame(avg_abs[avg_abs['Year'].between(1850,1975)][['Temp']])
# model = LinRegress(X['Year'], y['Temp'])
# yy= linear_func(X, *model._results.params)
# 
# fig.add_trace(go.Scatter(x = X['Year'], y = yy['Year'], mode = 'lines',
#                          showlegend=False, marker_color=light_red))
# 
# ####     REGRESSION     ####
# 
# X = pd.DataFrame(avg_abs[avg_abs['Year'].between(1975,2100)][['Year']])
# y = pd.DataFrame(avg_abs[avg_abs['Year'].between(1975,2100)][['Temp']])
# model = LinRegress(X['Year'], y['Temp'])
# yy= linear_func(X, *model._results.params)
# 
# fig.add_trace(go.Scatter(x = X['Year'], y = yy['Year'], mode = 'lines',
#                          showlegend=False, marker_color=light_red))
# 
# ####     1975 LINE     ####
# 
# fig.add_trace(go.Scatter( x=[1975, 1975], y=[13.5, 15.5], mode="lines",line=go.scatter.Line(color="gray"), showlegend=False))

In [44]:
fig = go.Figure()
fig.update_layout(title="Average Temporal House Price Over Years: 1963 - 2024", title_font_size = 20,
                  font=dict( family="Courier New, monospace", size=12,color="#7f7f7f"),
                  template = "ggplot2", hovermode= 'closest')
fig.update_xaxes(showline=True, linewidth=1, linecolor='gray')
fig.update_yaxes(showline=True, linewidth=1, linecolor='gray')

fig.add_trace(go.Scatter(x = year_avg['year'], y = year_avg['MSPUS'], mode = 'lines',
                         name = 'Avg. Temp', marker_color=red))

In [7]:
# regressions

X = pd.DataFrame(avg_abs[avg_abs['Year'].between(1975,2023)][['Year']])
y = pd.DataFrame(avg_abs[avg_abs['Year'].between(1975,2023)][['Temp']])

model = LinRegress(X['Year'], y['Temp'])

b,m = model._results.params
r = model.rsquared
t = model._results.tvalues[1]

print(f"y = {m}*x+{b}")
print(f"R² = {r}")
print(f"t = {t}")


y = 0.020302848639455914*x+-25.9525798044219
R² = 0.8921856134525356
t = 19.72141984812663


In [8]:
fig = make_subplots(rows=1, cols=1, insets=[{'cell': (1,1), 'l': 0.7, 'b': 0.3}], x_title = 'Temperature (°C)', subplot_titles=('Land Avg. Temperature vs. Time',  'Land Min. Temperature vs. Time', 'Land Max. Temperature vs. Time', 'Land & Ocean Avg. Temperature vs. Time'))
fig.update_layout(title="Average Temperatures Pre and Post 1975",font=dict( family="Courier New, monospace", size=15,color="#000000"),
                  template = "ggplot2", title_font_size = 20, hovermode= 'closest')

fig.add_trace(go.Box(x = avg_abs['Temp'], y = avg_abs['turnpoint'], boxpoints = 'all',jitter = 0.3,
                     pointpos = -1.6, marker_color = red, boxmean = True, showlegend=False),
              row = 1, col = 1)

fig.update_traces(orientation='h')
#pio.write_image(fig, "images/fig2.png", width=5*200, height=2*200, scale=.75)
fig

t-test

In [9]:
dfc = pd.read_csv('CSV/CO2 Concentration/CO2 Concentration.csv')
dfc.dropna(axis = 0, inplace = True)
dfc.drop(columns = 'Decimal Date', axis = 1, inplace = True)

dfc.info()

<class 'pandas.core.frame.DataFrame'>
Index: 702 entries, 2 to 708
Data columns (total 6 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   Year                               702 non-null    int64  
 1   Month                              702 non-null    int64  
 2   Carbon Dioxide (ppm)               702 non-null    float64
 3   Seasonally Adjusted CO2 (ppm)      702 non-null    float64
 4   Carbon Dioxide Fit (ppm)           702 non-null    float64
 5   Seasonally Adjusted CO2 Fit (ppm)  702 non-null    float64
dtypes: float64(4), int64(2)
memory usage: 38.4 KB


Carbon data setup

In [10]:
carbon_year = dfc.groupby(by = 'Year')[['Carbon Dioxide (ppm)']].mean().reset_index()
carbon_month = dfc.groupby(by = ['Year', 'Month'])[['Carbon Dioxide (ppm)']]


In [11]:
popt, pcov = curve_fit(exponenial_func, carbon_year['Year'], carbon_year['Carbon Dioxide (ppm)'], p0=(1, 1e-6, 1), maxfev=10000)

In [12]:
yy = exponenial_func(carbon_year['Year'], *popt)
a, b, c = popt
b*=-1
print(f"y = {a}*e^{b}*x + {c}")
rSquared = r2_score(carbon_year['Carbon Dioxide (ppm)'], yy)
print(f"R² = {rSquared}")

y = 6.14506622218523e-13*e^0.016418987055116834*x + 257.6808192876246
R² = 0.9993669394340178


In [13]:
X = carbon_year['Year']
y = carbon_year['Carbon Dioxide (ppm)']

data = pd.DataFrame({'x':X, 'y':y})
model = ols("y ~ x", data).fit()

print(model.summary())

yyy= linear_func(X, *model._results.params)

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.983
Model:                            OLS   Adj. R-squared:                  0.983
Method:                 Least Squares   F-statistic:                     3442.
Date:                Mon, 12 Aug 2024   Prob (F-statistic):           2.41e-53
Time:                        22:20:21   Log-Likelihood:                -159.51
No. Observations:                  60   AIC:                             323.0
Df Residuals:                      58   BIC:                             327.2
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept  -2701.1414     52.054    -51.891      0.0

In [14]:
fig = go.Figure()
fig.update_layout(title="Carbon Dioxide PPM per Year (1958 - 2017)", title_font_size = 20,
                  font=dict( family="Courier New, monospace", size=15,color="#000000"),
                  template = "ggplot2", hovermode= 'closest', legend=dict(
    yanchor="top",
    y=0.99,
    xanchor="left",
    x=0.01
))
fig.update_xaxes(showline=True, linewidth=1, linecolor='gray', title = 'Time (Year)')
fig.update_yaxes(showline=True, linewidth=1, linecolor='gray', title = 'Carbon Dioxide (PPM)')

fig.add_trace(go.Scatter(x = carbon_year['Year'], y = yy, mode = 'lines',
                         name = 'Exponential Regression Line', marker_color=light_blue))
fig.add_trace(go.Scatter(x = carbon_year['Year'], y = carbon_year['Carbon Dioxide (ppm)'], mode = 'markers',
                         showlegend=False, marker_color=red))

#pio.write_image(fig, "images/fig3.png", width=5*200, height=2*200, scale=.75)
fig

In [15]:
# regression

X = carbon_year[carbon_year['Year'].between(1958,2017)][['Carbon Dioxide (ppm)']].reset_index()
Y = avg_abs[avg_abs['Year'].between(1958,2017)][['Temp']].reset_index()

data = pd.DataFrame({'x':X['Carbon Dioxide (ppm)'], 'y':Y['Temp']})
model = ols("y ~ x", data).fit()

print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.909
Model:                            OLS   Adj. R-squared:                  0.907
Method:                 Least Squares   F-statistic:                     579.8
Date:                Mon, 12 Aug 2024   Prob (F-statistic):           6.95e-32
Time:                        22:20:22   Log-Likelihood:                 55.935
No. Observations:                  60   AIC:                            -107.9
Df Residuals:                      58   BIC:                            -103.7
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     10.4692      0.165     63.483      0.0

In [16]:
yy = linear_func(carbon_year['Carbon Dioxide (ppm)'], *model._results.params)

In [17]:
fig = go.Figure()
fig.update_layout(title="Global Land Avg. Temp vs. Carbon Dioxide <br> PPM per Year (1958 - 2017)", title_font_size = 30,
                  font=dict( family="Courier New, monospace", size=25,color="#000000"),
                  template = "ggplot2", hovermode= 'closest', legend=dict(
    yanchor="top",
    y=0.99,
    xanchor="left",
    x=0.01
))
fig.update_xaxes(showline=True, linewidth=1, linecolor='gray', title = 'Carbon Dioxide (PPM)')
fig.update_yaxes(showline=True, linewidth=1, linecolor='gray', title = 'Land Average Temperature (°C)')

fig.add_trace(go.Scatter(x = X['Carbon Dioxide (ppm)'], y=yy, mode = 'lines',
                         name = 'Linear Regression Line', marker_color=light_blue))
fig.add_trace(go.Scatter(x = X['Carbon Dioxide (ppm)'], y=Y['Temp'], mode = 'markers',
                         showlegend=False, marker_color=red))

#pio.write_image(fig, "images/fig4.png", width=5*200, height=2*200, scale=.75)
#pio.write_image(fig, "images/fig4poster.png", width=8.2*100, height=6.5*100, scale=.5)

fig

In [18]:
ttest_ind(carbon_year['Carbon Dioxide (ppm)'], avg_abs['Temp'], equal_var=False)

TtestResult(statistic=np.float64(96.94539495023938), pvalue=np.float64(9.175217942393006e-67), df=np.float64(59.00924008613353))

In [19]:
carboncorr = carbon_year[carbon_year['Year'].between(1958,2015)]
carboncorr = carboncorr.rename(columns={'Year':'Year', 'Carbon Dioxide (ppm)':'data'})
tempcorr = avg_abs[avg_abs['Year'].between(1958,2015)][['Year', 'Temp']]
tempcorr = tempcorr.rename(columns={'Temp':'data'})

pearsonr(tempcorr['data'], carboncorr['data'])

PearsonRResult(statistic=np.float64(0.947171959429479), pvalue=np.float64(2.466894933770841e-29))

In [20]:
data = { 
    "Years": [2017, 2018, 2019, 2020, 2021, 2022, 2023, 2024, 2025, 2026, 2027, 2028, 2029, 2030, 2031, 2032, 2033, 2034, 2035, 2036, 2037, 2038, 2039, 2040, 2041, 2042, 2043, 2044, 2045, 2046, 2047, 2048, 2049, 2050]
}

Model

Co2 v t: y = 6.14506622218523e-13\*e^0.016418987055116834*x + 257.6808192876246

Temp v Co2: y = 0.0112*x + 10.4692


In [21]:
def model(x):
    return 0.0112*(6.14506622218523*pow(10,-13)*pow(math.e, (0.016418987055116834*x)) + 257.6808192876246)+10.4692

datadf = pd.DataFrame(data)
aa = model(datadf["Years"])

In [22]:
Y = avg_abs[avg_abs['Year'].between(2000,2016)][['Temp']].reset_index()
X = avg_abs[avg_abs['Year'].between(2000,2017)][['Year']].reset_index()

In [23]:
aa.head(10)

0    15.016024
1    15.043518
2    15.071467
3    15.099878
4    15.128760
5    15.158120
6    15.187966
7    15.218306
8    15.249149
9    15.280502
Name: Years, dtype: float64

In [24]:
Y.loc[len(df.index)] = [2017, 15.016024]

In [25]:
fig = go.Figure()
fig.update_layout(title="Future Global Land Avg. Temperature Trends: 2000-2050", title_font_size = 20,
                  font=dict( family="Courier New, monospace", size=15,color="#000000"),
                  template = "ggplot2", hovermode= 'closest', legend=dict(
    yanchor="top",
    y=0.99,
    xanchor="left",
    x=0.01
))
fig.update_xaxes(showline=True, linewidth=1, linecolor='gray', title = 'Year')
fig.update_yaxes(showline=True, linewidth=1, linecolor='gray', title = 'Temperature')

fig.add_trace(go.Scatter(x = X['Year'], y = Y['Temp'], mode = 'lines',
                         name = 'Historic Temperature Data', marker_color=red))
fig.add_trace(go.Scatter(x = datadf["Years"], y = aa, name = 'Model Predicted Data', line=dict(
                          color=red, dash='dash')))

#pio.write_image(fig, "images/fig10.png", width=5*200, height=2*200, scale=.75)
# fig.update_layout(title="Future Global Land Avg. Temperature <br>Trends: 2000-2035", title_font_size = 30,
#                   font=dict( family="Courier New, monospace", size=25,color="#000000"),
#                   template = "ggplot2", hovermode= 'closest', legend=dict(
#     yanchor="top",
#     y=0.99,
#     xanchor="left",
#     x=0.01
# ))
#pio.write_image(fig, "images/fig10poster.png", width=8.2*100, height=6.5*100, scale=.5)
fig